# Ticket 4: Extracción de Características Simples

En este notebook se extraen features básicas de la señal EEG para preparar el terreno hacia modelos predictivos. Se calculará la potencia (energía) en las siguientes bandas de frecuencia:

- **Delta:** 0.5 - 4 Hz
- **Theta:** 4 - 8 Hz
- **Alpha:** 8 - 13 Hz
- **Beta:** 13 - 30 Hz

Para ello se utiliza la transformada de Fourier (a través de la función `welch` de SciPy) para obtener el espectro de potencia de cada canal. Luego se integra el área bajo la curva en cada banda para obtener el valor de energía, expresado en µV² (asumiendo que la unidad de la señal es microvoltios).

Cada fila de la tabla resultante corresponderá a un canal EEG (según el estándar 10-20) y cada columna representará la potencia en una banda de frecuencia.

In [1]:
# Importar las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import welch

print('Entorno configurado y librerías importadas correctamente.')

Entorno configurado y librerías importadas correctamente.


In [2]:
# Cargar el archivo v1p.mat
mat_data = loadmat('v1p.mat', squeeze_me=True)

# Mostrar las llaves disponibles en el archivo
print('Llaves en el archivo .mat:', mat_data.keys())

# Extraer la variable 'v1p'
data_array = mat_data['v1p']

print('Forma del array:', data_array.shape)
print('Tipo de datos:', data_array.dtype)

Llaves en el archivo .mat: dict_keys(['__header__', '__version__', '__globals__', 'v1p'])
Forma del array: (12258, 19)
Tipo de datos: int16


In [3]:
# Convertir la matriz en un DataFrame de pandas
df = pd.DataFrame(data_array)

# Asignar nombres de columnas de acuerdo con el estándar 10-20
channel_names = ['Fz', 'Cz', 'Pz', 'C3', 'T3', 'C4', 'T4', 'Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'P3', 'P4', 'T5', 'T6', 'O1', 'O2']

if df.shape[1] == len(channel_names):
    df.columns = channel_names
else:
    print('Advertencia: El número de columnas en el dataset no coincide con el número de canales esperados.')

print('Primeras 5 filas del DataFrame:')
print(df.head())

Primeras 5 filas del DataFrame:
    Fz   Cz   Pz   C3   T3   C4   T4  Fp1  Fp2    F3   F4   F7   F8   P3   P4  \
0   85 -407  200  191  420  457  310  310   16  1009  531  126  457  200  457   
1 -266  -55  -20  367  163  384  -20  310  494  1193  494  236  236  310  200   
2  -90  -19  126  437  420  568  347  457 -131  1156  384  384  494  384  494   
3  -90 -160  163  473  384  494  310  384  457  1340  494  420  310  420  273   
4 -301 -336  -20  473  200  531   89  420  200  1156  310  494  273  457  236   

    T5   T6   O1   O2  
0  384  -90  473  121  
1  457 -195  543   15  
2  531  -19  613  261  
3  531  -90  437  -19  
4  568 -160  578  121  


## Parámetros y Definición de Bandas de Frecuencia

Se definen las bandas de interés y la frecuencia de muestreo (128 Hz, según la documentación).

In [4]:
# Frecuencia de muestreo
fs = 128  # Hz

# Definir bandas de frecuencia (en Hz)
bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30)
}

print('Parámetros definidos:')
print(f'Frecuencia de muestreo: {fs} Hz')
print('Bandas de frecuencia:', bands)

Parámetros definidos:
Frecuencia de muestreo: 128 Hz
Bandas de frecuencia: {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30)}


## Función para Calcular la Potencia en una Banda de Frecuencia

Se utiliza la función `welch` para estimar la densidad espectral de potencia (PSD) de la señal. Luego, se integra el área bajo la curva (utilizando la regla del trapecio) en el rango de frecuencia correspondiente a cada banda.

In [5]:
def bandpower(data, fs, band, nperseg=256):
    """Calcula la potencia de la señal en una banda de frecuencia específica.
    
    Parámetros:
    - data: Señal unidimensional (array).
    - fs: Frecuencia de muestreo en Hz.
    - band: Tupla (f_low, f_high) que define la banda de interés.
    - nperseg: Número de muestras por segmento para welch.
    
    Retorna:
    - band_power: Potencia integrada en la banda (en µV², si la señal está en µV).
    """
    f, Pxx = welch(data, fs=fs, nperseg=nperseg)
    # Seleccionar frecuencias dentro de la banda
    freq_idx = np.logical_and(f >= band[0], f <= band[1])
    band_power = np.trapz(Pxx[freq_idx], f[freq_idx])
    return band_power

# Ejemplo: Calcular la potencia en la banda theta para el canal 'Fz'
# power_theta_Fz = bandpower(df['Fz'].values, fs, bands['theta'])
# print('Potencia banda theta para Fz:', power_theta_Fz, 'µV²')

## Extracción de Features para Todos los Canales

Para cada canal se calcula la potencia en cada banda (delta, theta, alpha y beta) y se resumen los resultados en una tabla.

In [6]:
# Crear una lista para almacenar los resultados
results = []

for channel in df.columns:
    channel_data = df[channel].values
    features = {'Canal': channel}
    
    # Calcular la potencia para cada banda
    for band_name, band_range in bands.items():
        power = bandpower(channel_data, fs, band_range, nperseg=256)
        features[f'Potencia_{band_name}'] = power
    
    results.append(features)

# Convertir los resultados en un DataFrame para visualizarlos
features_df = pd.DataFrame(results)

print('Tabla de características extraídas (potencia en µV²):')
print(features_df)

C:\Users\Luna\AppData\Local\Temp\ipykernel_7448\92541027.py:16: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  band_power = np.trapz(Pxx[freq_idx], f[freq_idx])


Tabla de características extraídas (potencia en µV²):
   Canal  Potencia_delta  Potencia_theta  Potencia_alpha  Potencia_beta
0     Fz   109265.653809    13519.541077     5118.315887    3295.481617
1     Cz    38420.385254    12905.592041     8500.407043   10504.804123
2     Pz    63465.504395     7656.492310     3874.831543    2719.671400
3     C3    22730.790771     8391.510986     5577.296021    8129.711044
4     T3     7931.026123     6505.762451     3447.874741    1536.809253
5     C4     9432.587769     7526.879456     5003.300140    1537.342149
6     T4     7740.396606     4720.436798     3728.219330    1891.740145
7    Fp1    22335.975342     7731.656982     5045.610107    3388.430733
8    Fp2     5090.586060     2723.075165     1840.601265    1520.216381
9     F3   124350.875488    45528.750977    24281.230225   53226.889771
10    F4    13944.471436     4430.856995     2647.535843    1542.630876
11    F7     5578.893982     2640.643494     1576.403145     963.499304
12    F8  

## Conclusiones e Interpretación

En este notebook se han extraído features simples a partir de las señales EEG. Para cada canal se ha calculado la potencia en las bandas delta, theta, alpha y beta. La tabla resultante resume estos valores y puede ser utilizada como input para modelos predictivos o para estudios comparativos.

**Consejo Profesional:**

La extracción de características es un paso esencial en la preparación de datos para inteligencia artificial. Es importante asegurarse de que los cálculos sean consistentes con las unidades esperadas (en este caso, µV²) y de interpretar correctamente los resultados. Ajusta los parámetros (por ejemplo, nperseg en la función welch) según la naturaleza y la duración de la señal para obtener estimaciones precisas. ¡Sigue explorando técnicas avanzadas para mejorar la robustez de tus features y la eficacia de tus modelos!